In [1]:
import json
import requests

--2025-07-10 13:36:20--  https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     671  --.-KB/s    in 0s      

2025-07-10 13:36:20 (36.5 MB/s) - ‘config.json’ saved [671/671]

7616042675:AAGf10TxZoIBLTUoBu4SecHl-aMuVFU5JiI


In [ ]:
!wget https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json

In [2]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def load_config(path="config.json"):
    with open(path, "r") as f:
        return json.load(f)

def send_email(subject, body):
    config = load_config()
    email_conf = config["email"]

    msg = MIMEMultipart()
    msg["From"] = email_conf["sender_email"]
    msg["To"] = email_conf["receiver_email"]
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP(email_conf["smtp_server"], email_conf["smtp_port"]) as server:
        server.starttls()
        server.login(email_conf["sender_email"], email_conf["sender_password"])
        server.send_message(msg)

    print("Email is sent successfully.")

In [3]:
send_email("Job Alert Test", "Ceci n'est pas un bot.")

Email is sent successfully.


In [4]:
def send_telegram(message):
  config = load_config()
  token = config["telegram"]["bot_token"]
  chat_id = config["telegram"]["chat_id"]

  url = f"https://api.telegram.org/bot{token}/sendMessage"
  payload = {
      "chat_id" : chat_id,
      "text": message
  }

  response = requests.post(url, data= payload)

  if response.status_code == 200:
    print("Message is sent successfully.")
  else:
    print(f"Error: {response.status_code} - {response.text}")


In [5]:
send_telegram("Ceci n'est pas un bot.")

Message is sent successfully.


In [6]:
def notification(subject, message):
  try:
    send_email(subject, message)
  except Exception as excep:
    print(f"Email Error: {excep}")

  try:
    send_telegram(message)
  except Exception as excep:
    print(f"Telegram Error: {excep}")

In [7]:
notification("New job post", "New job arrived")

Email is sent successfully.
Message is sent successfully.


In [9]:
def remote_jobs():
    url = "https://remoteok.com/api"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        jobs = response.json()
        return jobs[1:]
    else:
        print(f"Hata: {response.status_code}")
        return []

In [10]:
jobs = remote_jobs()
for job in jobs[:5]:
    print(job["position"], "-", job.get("company", ""))

Customer Onboarding & Support Specialist - Gymflow
Senior TS React Developer - Prezly
AI Driven Engineer Ruby - CloudWalk
Senior Security Engineer - Immutable
Software Engineer - Loancrate


In [11]:
def relevant_job(job, config):
    title = job.get("position", "").lower()
    description = job.get("description", "").lower()

    # Anahtar kelime eşleşmesi (pozisyon)
    if not any(kw.lower() in title for kw in config["search_keywords"]):
        return False

    # Visa/relocation kontrolü (açıklama)
    if not any(kw.lower() in description for kw in config["visa_keywords"]):
        return False

    return True

In [12]:
jobs = remote_jobs()
config = load_config()

for job in jobs:
    if relevant_job(job, config):
        message = f" {job['position']} @ {job.get('company', 'Unknown')}\n🌍 {job.get('location', 'Remote')}\n🔗 {job.get('url', '')}"
        notification("Yeni İş İlanı!", message)